# PyFP - The `fn.py` Package

Fn.py: enjoy FP in Python

Documentation:
* https://github.com/kachayev/fn.py

# Introduction

Despite the fact that Python is not pure-functional programming language, it's multi-paradigm PL and it gives you enough freedom to take credits from functional programming approach. There are theoretical and practical advantages to the functional style:

* Formal provability
* Modularity
* Composability
* Ease of debugging and testing

`fn.py` library provides you with missing "batteries" to get maximum from functional approach even in mostly-imperative program.

More about functional approach from my Pycon UA 2012 talks: [Functional Programming with Python.](http://kachayev.github.io/talks/uapycon2012/index.html#/)

# Installation
To install fn.py, simply:

```
$ pip install fn

```

Or, if you absolutely must:

```
$ easy_install fn

```

You can also build library from source

```
$ git clone https://github.com/kachayev/fn.py.git
$ cd fn.py
$ python setup.py install
```

# Scala-style lambdas definition

In [4]:
from fn import _
from fn.op import zipwith
from itertools import repeat

assert list(map(_ * 2, range(5))) == [0,2,4,6,8]
assert list(filter(_ < 10, [9,10,11])) == [9]
assert list(zipwith(_ + _)([0,1,2], repeat(10))) == [10,11,12]

If you are not sure, what your function is going to do, you can print it:

In [5]:
from fn import _

print (_ + 2) # "(x1) => (x1 + 2)"
print (_ + _ * _) # "(x1, x2, x3) => (x1 + (x2 * x3))"

(x1) => (x1 + 2)
(x1, x2, x3) => (x1 + (x2 * x3))


`_` will fail with `ArityError` (`TypeError` subclass) on inaccurate number of passed arguments.

# Persistent data structures
**Attention:** Persistent data structures are under active development.

Persistent data structure is a data structure that always preserves the previous version of itself when it is modified (more formal information on Wikipedia). Each operation with such data structure yields a new updated structure instead of in-place modification (all previous versions are potentially available or GC-ed when possible).

Lets take a quick look:

In [6]:
from fn.immutable import SkewHeap

s1 = SkewHeap(10)
s2 = s1.insert(20)
s2

In [7]:
s3 = s2.insert(30)
s3  # <-- other object

In [8]:
s3.extract()

(10, <fn.immutable.heap.SkewHeap at 0x106cc6a70>)

In [9]:
s3.extract() # <-- s3 isn't changed

(10, <fn.immutable.heap.SkewHeap at 0x106cc6d48>)

# Streams and infinite sequences declaration
Lazy-evaluated Scala-style streams. Basic idea: evaluate each new element "on demand" and share calculated elements between all created iterators. Stream object supports << operator that means pushing new elements when it's necessary.

Simplest cases:

In [10]:
from fn import Stream

s = Stream() << [1,2,3,4,5]
assert list(s) == [1,2,3,4,5]
assert s[1] == 2
assert list(s[0:2]) == [1,2]

s = Stream() << range(6) << [6,7]
assert list(s) == [0,1,2,3,4,5,6,7]

def gen():
    yield 1
    yield 2
    yield 3

s = Stream() << gen << (4,5)
assert list(s) == [1,2,3,4,5]

Lazy-evaluated stream is useful for infinite sequences, i.e. fibonacci sequence can be calculated as:

In [11]:
from fn import Stream
from fn.iters import take, drop, map
from operator import add

f = Stream()
fib = f << [0, 1] << map(add, f, drop(1, f))

assert list(take(10, fib)) == [0,1,1,2,3,5,8,13,21,34]
assert fib[20] == 6765
assert list(fib[30:35]) == [832040,1346269,2178309,3524578,5702887]

# Trampolines decorator
fn.recur.tco is a workaround for dealing with TCO without heavy stack utilization. Let's start from simple example of recursive factorial calculation:

In [12]:
def fact(n):
    if n == 0: return 1
    return n * fact(n-1)

This variant works, but it's really ugly. Why? It will utilize memory too heavy cause of recursive storing all previous values to calculate final result. If you will execute this function with big n (more than sys.getrecursionlimit()) CPython will fail with

In [13]:
import sys
fact(sys.getrecursionlimit() * 2)

RecursionError: maximum recursion depth exceeded in comparison

Which is good, cause it prevents you from terrible mistakes in your code.

How can we optimize this solution? Answer is simple, lets transform function to use tail call:

In [14]:
def fact(n, acc=1):
    if n == 0: return acc
    return fact(n-1, acc*n)

Why this variant is better? Cause you don't need to remember previous values to calculate final result. More about tail call optimization on [Wikipedia](https://en.wikipedia.org/wiki/Tail_call). 

But... Python interpreter will execute this function the same way as previous one, so you won't win anything.

`fn.recur.tco` gives you mechanism to write "optimized a bit" tail call recursion (using "trampoline" approach):

In [15]:
from fn import recur

@recur.tco
def fact(n, acc=1):
    if n == 0: return False, acc
    return True, (n-1, acc*n)

@recur.tco is a decorator that execute your function in while loop and check output:

* `(False, result)` means that we finished
* `(True, args, kwargs)` means that we need to call function again with other arguments
* `(func, args, kwargs)` to switch function to be executed inside while loop

The last variant is really useful, when you need to switch callable inside evaluation loop. Good example for such situation is recursive detection if given number is odd or even:

In [17]:
from fn import recur

@recur.tco
def even(x):
    if x == 0: return False, True
    return odd, (x-1,)

@recur.tco
def odd(x):
    if x == 0: return False, False
    return even, (x-1,)

print(even(100000))

True


# High-level operations with functions
`fn.F` is a useful function wrapper to provide easy-to-use partial application and functions composition.

In [18]:
from fn import F, _
from operator import add, mul

# F(f, *args) means partial application
# same as functools.partial but returns fn.F instance
assert F(add, 1)(10) == 11

# F << F means functions composition,
# so (F(f) << g)(x) == f(g(x))
f = F(add, 1) << F(mul, 100)
assert list(map(f, [0, 1, 2])) == [1, 101, 201]
assert list(map(F() << str << (_ ** 2) << (_ + 1), range(3))) == ["1", "4", "9"]

It also give you move readable in many cases "pipe" notation to deal with functions composition:

In [19]:
from fn import F, _
from fn.iters import filter, range

func = F() >> (filter, _ < 6) >> sum
assert func(range(10)) == 15

You can find more examples for compositions usage in `fn._` implementation source code.

`fn.op.apply` executes given function with given positional arguments in list (or any other iterable). `fn.op.flip` returns you function that will reverse arguments order before apply.

In [20]:
from fn.op import apply, flip
from operator import add, sub

assert apply(add, [1, 2]) == 3
assert flip(sub)(20,10) == -10
assert list(map(apply, [add, mul], [(1,2), (10,20)])) == [3, 200]

`fn.op.foldl` and `fn.op.foldr` are folding operators. Each accepts function with arity 2 and returns function that can be used to reduce iterable to scalar: from left-to-right and from right-to-left in case of foldl and foldr respectively.

In [21]:
from fn import op, _

folder = op.foldr(_ * _, 1)
assert 6 == op.foldl(_ + _)([1,2,3])
assert 6 == folder([1,2,3])

Use case specific for right-side folding is:

In [22]:
from fn.op import foldr, call

assert 100 == foldr(call, 0 )([lambda s: s**2, lambda k: k+10])
assert 400 == foldr(call, 10)([lambda s: s**2, lambda k: k+10])

# Function currying
fn.func.curried is a decorator for building curried functions, for example:

In [24]:
from fn.func import curried
@curried
def sum5(a, b, c, d, e):
    return a + b + c + d + e

print(sum5(1)(2)(3)(4)(5))
print(sum5(1, 2, 3)(4, 5))

15
15


# Functional style for error-handling
`fn.monad.Option` represents optional values, each instance of `Option` can be either instance of `Full` or `Empty`. It provides you with simple way to write long computation sequences and get rid of many `if/else` blocks. See usage examples below.

Assume that you have Request class that gives you parameter value by its name. To get uppercase notation for non-empty striped value:

In [25]:
class Request(dict):
    def parameter(self, name):
        return self.get(name, None)

r = Request(testing="Fixed", empty="   ")
param = r.parameter("testing")
if param is None:
    fixed = ""
else:
    param = param.strip()
    if len(param) == 0:
        fixed = ""
    else:
        fixed = param.upper()

Hmm, looks ugly.. Update code with `fn.monad.Option`:

In [29]:
from operator import methodcaller
from fn.monad import optionable

class Request(dict):
    @optionable
    def parameter(self, name):
        return self.get(name, None)

r = Request(testing="Fixed", empty="   ")
fixed = (
         r.parameter("testing")
          .map(methodcaller("strip"))
          .filter(len)
          .map(methodcaller("upper"))
          .get_or("")
        )